In [1]:
!pip install torchvision
!pip install diffusers
!pip install transformers
!pip install accelerate
!pip install ftfy
!pip install gradio
!pip install imageio
!pip install modin[all]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 852.0/852.0 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 32.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 1.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.8 MB/s eta 0:00:00
Looking in indexes: http

In [2]:
from diffusers import StableDiffusionInpaintPipeline
import gradio as gr
import numpy as np
import imageio
from PIL import Image
import torch
import modin.pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionInpaintPipeline.from_pretrained("stabilityai/stable-diffusion-2-inpainting", safety_checker=None)
pipe = pipe.to(device)

def resize(height,img):
  baseheight = height
  img = Image.open(img)
  hpercent = (baseheight/float(img.size[1]))
  wsize = int((float(img.size[0])*float(hpercent)))
  img = img.resize((wsize,baseheight), Image.LANCZOS)
  return img

def predict(source_img, prompt, negative_prompt):
    imageio.imwrite("data.png", source_img["image"])
    imageio.imwrite("data_mask.png", source_img["mask"])
    src = resize(512, "data.png")
    src.save("src.png")
    mask = resize(512, "data_mask.png")  
    mask.save("mask.png")
    image = pipe(prompt=prompt, negative_prompt=negative_prompt, image=src, mask_image=mask, num_inference_steps=20).images[0]
    return image

title="Stable Diffusion 2.0 Inpainting CPU"
description="Inpainting with Stable Diffusion 2.0 <br />Warning: Slow process... ~10 min inference time.<br> <b>Please use 512x512 or 768x768 square .png image as input to avoid memory error!!!</b>"
gr.Interface(fn=predict, inputs=[gr.Image(source="upload", type="numpy", tool="sketch", elem_id="source_container"), gr.Textbox(label='What you want the AI to Generate, 77 Token limit'), gr.Textbox(label='What you Do Not want the AI to generate')], outputs='image', title=title, description=description, article = "Code Monkey: <a href=\"https://huggingface.co/Manjushri\">Manjushri</a>").launch(max_threads=True, debug=True)

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
